# Kodiranje izračunljivih funkcija

Nula $Z : N^k \rightarrow N$, $Z(\overrightarrow{x}) = 0$

In [238]:
class zero(object):
    code = 0
    def __call__(self,*args):
        return 0
    def __str__(self):
        return 'Z'

Z = zero()
print(Z(3,2,5))
print(Z)

0
Z


Sljedbenik $S : N \rightarrow N$, $S(x) = x+1$

In [239]:
class successor(object):
    code = 1
    def __call__(self,n):
        return n+1
    def __str__(self):
        return 'S'
    
S = successor()
S(7)

8

Projekcija $I : N^k \rightarrow N$, $I(n,\overrightarrow{x}) = x_n$, $1 \leq n \leq k$

In [345]:
class projection(object):
    code = 2
    def __init__(self,n=1):
        self.n = n
    def __call__(self,*args):
        return args[self.n-1]
    def __str__(self):
        return 'I'+str(self.n)

I2 = projection(2)
print(I2(1,2,3,4))
print(I2)
    

2
I2


In [397]:
primes = [7,11,13,17,19]
# TODO: Implement sieve of Atkin, just for fun

Želimo označiti kopoziciju, primitivnu rekurziju i minimizaciju nekim posebnim kodovima

- kompozicija -> 2
- primitivna rekurzija -> 3
- minimizacija -> 5

### Kompozicija

In [402]:
import operator
from functools import reduce

def compose(g, *hs):
    hs_num = len(hs)
    g_params = [None]*hs_num
    class composition(object):
        #TODO: unique primes to represent ¤
        code = 2 * primes[0]**(1+g.code) * reduce(operator.mul, (primes[ob_i+1]**(1+ob.code) for ob_i,ob in enumerate(hs)), 1)
        def __call__(self,*args):
            for i in range(0,hs_num):
                g_params[i] = hs[i](*args)
                #print("g_params[{}]: {}".format(i, g_params[i]))
            return g(*g_params)
        def __str__(self):
            name = '('+str(g)+'¤'
            if(hs_num>1):
                name += '('
            for i in range(0,hs_num-1):
                name += str(hs[i])+','
            name += str(hs[-1])
            if(hs_num>1):
                name += ')'
            return name+')'
    return composition()
    

In [375]:
test = compose(I2,S,compose(S,Z))
print(test(11))
print(test.code)
print(test)

1
87890625000
(I2¤(S,(S¤Z)))


### Primitivna rekurzija

In [376]:
def prim_recurse(g, h):
    class prim_recursion(object):
        #TODO: unique primes to represent pr
        code = 1 #primes[0]**(1+g.code) * primes[1]**(1+h.code) -> something is wrong ?:/?
        def __call__(self,*args):
            # po definiciji
            if len(args) == 1:
                # degenerirana verzija
                arg = args[0]
                if arg == 0:
                    return g()
                return h(arg-1,prim_recurse(g,h)(arg-1))
            # ne-degenerirana verzija
            if args[-1] == 0:
                return g(*args[:-1])
            return h(*args[:-1],args[-1]-1,prim_recurse(g,h)(*args[:-1],args[-1]-1))
        def __str__(self):
            return '('+str(g)+"pr"+str(h)+')'
    return prim_recursion()

In [377]:
I1 = projection(1)
I3 = projection(3)
add = prim_recurse(I1,compose(S,I3))
print(add)
print("9+11="+str(add(9,11)))

pd = prim_recurse(Z,I1)
#print(pd.code)
sub = prim_recurse(I1,compose(pd,I3))
print("5-2="+str(sub(5,2)))

(I1pr(S¤I3))
9+11=20
5-2=3


### Minimizacija

In [396]:
def minimize(g):
    class minimization(object):
        #TODO: unique primes to represent min
        code = 5 * primes[0]**(1+g.code)
        def __call__(self,*args):
            y = 0
            # po definiciji
            while g(*args,y) != 0: 
                y += 1
            return y
        def __str__(self):
            return "min("+str(g)+')'
    return minimization()

In [394]:
simple = minimize(sub)
print(simple)
print(simple(3))

min((I1pr((ZprI1)¤I3)))
3


In [416]:
I = projection()
f_dict = [Z,S,I]

def from_code(n):
    name = ""
    if n%2 == 0:
        name += "compose("
        n //= 2
    g_code = 0
    h_code = 0
    while n%7 == 0:
        n //= 7
        g_code += 1
    if g_code < 4:
        name += str(f_dict[g_code-1])
    name += ','
    while n%11 == 0:
        n //= 11
        h_code += 1
    if h_code < 4:
        name += str(f_dict[h_code-1])
    else:
        name += from_code(h_code-1)
    return name+')'

In [419]:
comp_test = compose(S,Z)
print(comp_test(7))
print(comp_test.code)
print(from_code(comp_test.code))

ct2 = compose(S,comp_test)
print(ct2(7))
print(ct2.code)
print(from_code(ct2.code))

1
1078
compose(S,Z)
2
450747536079067618000970174086475231404621651640367853331219740233193086107530014075453409250847362533983579817930197318632368458501500373415829296947592164218812515017427117937014772429997582521306131721771968018230506648797787492068906882619472326080181255182867076236711363466821742955376851697137436331974917980477961498451287858256592880796370318884159937441899876783530535026297737441178193630079121869580997110367099846705116627510831768613375929692879931088206702168806333330884206344426036836268322060937934079242487698520917712194425568269487275460770102728738903253043515615492624801632414423241921416712584830120607768168811493058325211984404477266618032729623094600140072176038649299195182146984246739965921386355282565263313276112226011478360917826218223446905871684162098331127974439125613637634174393090349247249156308320287041117141903010599991253578683088910271423633842973117272616090024083922343039695814061612530911792681363771961606311958854015195553130967